In [142]:
import pickle
import numpy as np
import torch
import matplotlib.pyplot as plt
import os

In [143]:
def load_pickle(file_path):
    try:
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
            return data
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [144]:
def save_as_pickle(data, filename, directory='/Users/qiyangyan/Desktop/Diffusion/Demonstration'):
    os.makedirs(directory, exist_ok=True)
    full_path = os.path.join(directory, filename)

    try:
        with open(full_path, 'wb') as file:
            pickle.dump(data, file)
        # print(full_path)
        print(f"Data successfully saved to {full_path}")
    except Exception as e:
        print(f"An error occurred while saving data: {e}")

In [145]:
pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/bigSteps_10k_demos_random_slide'  # Training
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/bigSteps_10k_demos_random_slide_testDataset'  # Testing
data = load_pickle(pickle_file_path)

In [146]:
print(data.keys())

In [147]:
rotation_start_pose_prediction_dataset = {
    'observations': [],
    'actions': [],
    'desired_goals': [],
    'next_observations': [],
    'sampled_desired_goals': [],
    'rewards': [],
    'terminals': []
}
for i, item in enumerate(data['terminals']):
    if item == 1:
        # predict based on the desired goal
        rotation_start_pose_prediction_dataset['observations'].append(data['desired_goals'][i])  
        rotation_start_pose_prediction_dataset['actions'].append(data['observations'][i][8:15])

for key in rotation_start_pose_prediction_dataset.keys():
    rotation_start_pose_prediction_dataset[key] = np.array(rotation_start_pose_prediction_dataset[key])
    print(np.shape(rotation_start_pose_prediction_dataset[key]))

In [148]:
np.shape(torch.cat([torch.tensor(rotation_start_pose_prediction_dataset['desired_goals']), torch.tensor(rotation_start_pose_prediction_dataset['observations'])], dim=1))

# Visualisation

In [191]:
# Plot target pose region
def quaternion_to_euler(w1, w2, w3, w4):
    # Quaternion to Euler angles (yaw, pitch, roll)
    
    # Yaw (z-axis rotation)
    siny_cosp = 2 * (w4 * w3 + w1 * w2)
    cosy_cosp = 1 - 2 * (w2 * w2 + w3 * w3)
    yaw = np.arctan2(siny_cosp, cosy_cosp)
    
    # Pitch (y-axis rotation)
    sinp = 2 * (w4 * w2 - w3 * w1)
    if abs(sinp) >= 1:
        pitch = np.sign(sinp) * np.pi / 2  # use 90 degrees if out of range
    else:
        pitch = np.arcsin(sinp)
    
    # Roll (x-axis rotation)
    sinr_cosp = 2 * (w4 * w1 + w2 * w3)
    cosr_cosp = 1 - 2 * (w1 * w1 + w2 * w2)
    roll = np.arctan2(sinr_cosp, cosr_cosp)
    
    yaw = np.degrees(yaw)
    pitch = np.degrees(pitch)
    roll = np.degrees(roll)
    
    return yaw, pitch, roll

plt.figure()
count = 0

# pose_list = rotation_start_pose_prediction_dataset['actions']
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/bigSteps_10k_demos_random_slide'  # Training
pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/bigSteps_10000demos_slide'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF-1135demos'
data = load_pickle(pickle_file_path)
pose_list = data['desired_goals'][:, :7]
# print(data['desired_goals'][:2])

angle_old = 0
for i in range(1):
    # Extract x, y, z and quaternion angle from the desired goals
    goal = pose_list[i]
    x, y, z, w1, w2, w3, w4 = goal

    # Convert quaternion to yaw angle
    angle = quaternion_to_euler(w1, w2, w3, w4)[2]
    if angle == angle_old:
        continue
    else:
        angle_old = angle

    # Scatter plot for x and y
    plt.scatter(x, y, )
    
    print(quaternion_to_euler(w1, w2, w3, w4))
    # Calculate direction for quiver (assuming angle is in radians)
    u = np.cos(angle)
    v = np.sin(angle)
    
    # Quiver for the orientation
    plt.quiver(x, y, u, v, angles='xy', scale_units='xy', scale=500, color='b')

    # if count > 10:
    #     break

print("Number of terminal states:", count)

plt.xlabel("y", fontsize='large', fontweight='bold')
plt.ylabel("x", fontsize='large', fontweight='bold')
plt.title("Target Position Distribution", fontweight='bold')
plt.grid(which='major')


# Training Dataset

In [150]:
# save_as_pickle(rotation_start_pose_prediction_dataset, 'rotation_start_pose_prediction_dataset')

# Testing Dataset

In [151]:
# save_as_pickle(rotation_start_pose_prediction_dataset, 'rotation_start_pose_prediction_dataset_testing')